In [2]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
from textwrap import dedent
import warnings
warnings.filterwarnings('ignore')
load_dotenv()
from urllib.parse import urlparse
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool)

In [10]:

azure_llm = AzureChatOpenAI(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.environ.get("AZURE_OPENAI_VERSION")
)


In [21]:
links = '''https://en.wikipedia.org/wiki/Paris_syndrome
https://writersblockmagazine.com/2021/11/23/paris-syndrome-romanticizing-your-life-and-changes
https://www.outlooktraveller.com/explore/inspiration/what-is-paris-syndrome
https://www.sbs.com.au/news/article/paris-syndrome-culture-shock-sickness-sends-japanese-tourists-packing/t8a332he2
https://www.independent.co.uk/travel/news-and-advice/what-is-paris-syndrome-b2477677.html
https://www.islands.com/1579130/what-is-paris-syndrome
https://themaddifoundation.com/wp-content/uploads/2017/12/dailypost-2.pdf
https://www.livescience.com/what-is-paris-syndrome
https://wanderdolls.com/paris-syndrome-bs
https://www.tripzilla.com/the-paris-syndrome-why-asians-suffer-from-it/127572
https://www.paristopten.com/what-is-paris-syndrome
https://travelbetweenthepages.com/2024/01/08/paris-syndrome
https://www.myfrenchlife.org/2013/08/19/so-french-so-sick-the-paris-syndrome
https://en.wikipedia.org/wiki/List_of_diplomatic_missions_in_Paris
https://findahelpline.com/countries/fr/topics/depression
https://findahelpline.com/countries/fr
https://soshelpline.org/mental-health-resources/'''.split('\n')


for e,link in enumerate(links):
    wikitool = ScrapeWebsiteTool(website_url=link)
    text = wikitool.run()
    parsed_url = urlparse(link)
    filename = parsed_url.path.replace('/', '-')
    with open(f'./scraped_data/{filename}.txt', 'w', encoding="utf-8") as f:
        f.write(text)
    
    print(f'{e} completed')

In [22]:

from crewai_tools import DirectorySearchTool


# Example 1: Initialize the tool without any parameters to scrape the current page it navigates to
search_tool = SerperDevTool(n_results=3)
docs_tool = DirectoryReadTool(directory='./scraped_data')
file_reader = FileReadTool()

scraper_tools = []
for e, link in enumerate(links):
    scraper_tools.append(ScrapeWebsiteTool(website_url=link))



In [15]:
general_health_advisor = Agent(
    role = "Mental Health Advisor",
    goal = "To provide mental health advice to the travelers suffering from paris syndrome",
    backstory = """You are a specialist who has knowledge of cognitive behavioural therapy, meditation techniques \n
        mindfulness practices and other therapeutic methods""",
    verbose=True,
    llm=azure_llm,
    max_iter=5,
    allow_delegation=True
)

def support_task( agent, origin, destination, answers, tool_list):
    task = Task(description = f"""I will provide you with an individual looking for guidance and \n
        advice on managing their emotions, stress, anxiety and other mental health issues arising due to Paris syndrome. \n
        Use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, \n
        and other therapeutic methods in order to create strategies that the individual can implement in order \n
        to improve their overall wellbeing.\n
        Use the {answers} to provide the best advice. 
                    
        Origin: {origin}
        Destination: {destination}
        """,
        expected_output=dedent("""
        A summary of the conversation highlighting:
        1. Key concerns expressed by the user.
        2. Emotional reassurance provided.
        3. Recommended coping strategies.
        4. Also, recommend any mental helplines in the {destination} if you have the correct information. \n
        Otherwise, do not provide any fake numbers.
        """),
        agent=agent,
        tools=tool_list
        )
    
    return task

In [16]:
answers = [("Expectations", "I had high expectations of Paris before arriving, \
    especially for visiting iconic landmarks like the Eiffel Tower and the Louvre Museum."),
("Experiences" ,"My experiences in Paris have been mixed, with some aspects being disappointing, \
    such as the crowds and long lines, but I have also discovered some hidden gems and enjoyed exploring lesser-known neighborhoods."),
("Emotional State", "I am feeling a bit overwhelmed and disoriented during my visit to Paris, \
    and have experienced some anxiety and homesickness."),
("Social Interactions", "My interactions with locals and other tourists have been mostly positive, \
    but I have faced some language barriers and cultural misunderstandings."),
("Coping Strategies", "To cope with my feelings of disappointment and distress, \
I have been focusing on the positive aspects of my trip and taking breaks from the crowds and noise."),
 ("Support Systems", "Unfortunately, I do not have any friends or family in Paris to talk to about my feelings."),
 ("Duration", "I plan to stay in Paris for another week before moving on to other destinations."),
 ("Health and Safety", "I have not needed to seek any medical or psychological assistance during my trip, \
     and I feel safe and secure in my current accommodation and surroundings."),
 ("Reflection" ,"Looking back, I think I could have prepared better for my trip to Paris \
     by doing more research on the city and its customs.\
     My advice to someone planning a trip to Paris would be to set realistic expectations and \
    be prepared for the crowds and noise."),
 ("Personal Background","This is not my first time traveling abroad, but it is my first time visiting Europe. In the past, I have enjoyed exploring new cultures and trying new foods, \
     but have also experienced culture shock and homesickness during my travels.")]
 

In [10]:
task = support_task( general_health_advisor, 'chennai', 'paris', answers, tool_list=[])

crew = Crew(
    agents=[general_health_advisor],
    tasks=[task],
    verbose=2
)

result = crew.kickoff()
print(result)

2024-06-01 07:31:44,334 - 72644 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Mental Health Advisor
 [INFO]: == Starting Task: I will provide you with an individual looking for guidance and 

        advice on managing their emotions, stress, anxiety and other mental health issues arising due to Paris syndrome. 

        Use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, 

        and other therapeutic methods in order to create strategies that the individual can implement in order 

        to improve their overall wellbeing.

        Use the [('Expectations', 'I had high expectations of Paris before arriving,     especially for visiting iconic landmarks like the Eiffel Tower and the Louvre Museum.'), ('Experiences', 'My experiences in Paris have been mixed, with some aspects being disappointing,     such as the crowds and long lines, but I have also discovered some hidden gems and enjoyed exploring lesser-known neighborhoods.'), ('Emotional State', 'I am feeling a bit overwhelmed and di

In [11]:
task = support_task( general_health_advisor, 'chennai', 'paris', answers, tool_list=[search_tool])

crew = Crew(
    agents=[general_health_advisor],
    tasks=[task],
    verbose=2
)

result = crew.kickoff()
print(result)

2024-06-01 07:32:03,375 - 72644 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Mental Health Advisor
 [INFO]: == Starting Task: I will provide you with an individual looking for guidance and 

        advice on managing their emotions, stress, anxiety and other mental health issues arising due to Paris syndrome. 

        Use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, 

        and other therapeutic methods in order to create strategies that the individual can implement in order 

        to improve their overall wellbeing.

        Use the [('Expectations', 'I had high expectations of Paris before arriving,     especially for visiting iconic landmarks like the Eiffel Tower and the Louvre Museum.'), ('Experiences', 'My experiences in Paris have been mixed, with some aspects being disappointing,     such as the crowds and long lines, but I have also discovered some hidden gems and enjoyed exploring lesser-known neighborhoods.'), ('Emotional State', 'I am feeling a bit overwhelmed and di

In [17]:
task = support_task( general_health_advisor, 'chennai', 'paris', answers, tool_list=[docs_tool, file_reader])

crew = Crew(
    agents=[general_health_advisor],
    tasks=[task],
    verbose=2
)

result = crew.kickoff()
print(result)

2024-06-01 21:06:49,327 - 44664 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Mental Health Advisor
 [INFO]: == Starting Task: I will provide you with an individual looking for guidance and 

        advice on managing their emotions, stress, anxiety and other mental health issues arising due to Paris syndrome. 

        Use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, 

        and other therapeutic methods in order to create strategies that the individual can implement in order 

        to improve their overall wellbeing.

        Use the [('Expectations', 'I had high expectations of Paris before arriving,     especially for visiting iconic landmarks like the Eiffel Tower and the Louvre Museum.'), ('Experiences', 'My experiences in Paris have been mixed, with some aspects being disappointing,     such as the crowds and long lines, but I have also discovered some hidden gems and enjoyed exploring lesser-known neighborhoods.'), ('Emotional State', 'I am feeling a bit overwhelmed and di

In [ ]:
task = support_task( general_health_advisor, 'chennai', 'paris', answers, tool_list=scraper_tools)

crew = Crew(
    agents=[general_health_advisor],
    tasks=[task],
    verbose=2
)

result = crew.kickoff()
print(result)